In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-06 01:40:52.819928: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=100)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_6_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_6_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

Epoch 21/1000
1/1 [==============================] - 0s 37ms/step - loss: 24.8936 - mean_squared_error: 24.8936 - rmse: 4.9894 - mean_absolute_error: 3.8511 - val_loss: 20.7374 - val_mean_squared_error: 20.7374 - val_rmse: 4.5538 - val_mean_absolute_error: 3.6565 - lr: 0.0010
Epoch 22/1000
1/1 [==============================] - 0s 58ms/step - loss: 24.2911 - mean_squared_error: 24.2911 - rmse: 4.9286 - mean_absolute_error: 3.8104 - val_loss: 21.3370 - val_mean_squared_error: 21.3370 - val_rmse: 4.6192 - val_mean_absolute_error: 3.7076 - lr: 0.0010
Epoch 23/1000
1/1 [==============================] - 0s 34ms/step - loss: 23.6722 - mean_squared_error: 23.6722 - rmse: 4.8654 - mean_absolute_error: 3.8451 - val_loss: 20.7247 - val_mean_squared_error: 20.7247 - val_rmse: 4.5524 - val_mean_absolute_error: 3.6542 - lr: 0.0010
Epoch 24/1000
1/1 [==============================] - 0s 32ms/step - loss: 28.0581 - mean_squared_error: 28.0581 - rmse: 5.2970 - mean_absolute_error: 4.2555 - val_loss: 

Epoch 51/1000
1/1 [==============================] - 0s 44ms/step - loss: 19.5589 - mean_squared_error: 19.5589 - rmse: 4.4225 - mean_absolute_error: 3.3930 - val_loss: 8.1379 - val_mean_squared_error: 8.1379 - val_rmse: 2.8527 - val_mean_absolute_error: 2.1170 - lr: 0.0010
Epoch 52/1000
1/1 [==============================] - 0s 44ms/step - loss: 18.6875 - mean_squared_error: 18.6875 - rmse: 4.3229 - mean_absolute_error: 3.5118 - val_loss: 8.5717 - val_mean_squared_error: 8.5717 - val_rmse: 2.9277 - val_mean_absolute_error: 2.2108 - lr: 0.0010
Epoch 53/1000
1/1 [==============================] - ETA: 0s - loss: 20.3282 - mean_squared_error: 20.3282 - rmse: 4.5087 - mean_absolute_error: 3.6233
Epoch 00053: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1/1 [==============================] - 0s 32ms/step - loss: 20.3282 - mean_squared_error: 20.3282 - rmse: 4.5087 - mean_absolute_error: 3.6233 - val_loss: 9.4657 - val_mean_squared_error: 9.4657 - val_rmse: 3.0766 - va

Epoch 79/1000
1/1 [==============================] - 0s 36ms/step - loss: 13.8823 - mean_squared_error: 13.8823 - rmse: 3.7259 - mean_absolute_error: 2.9561 - val_loss: 9.6627 - val_mean_squared_error: 9.6627 - val_rmse: 3.1085 - val_mean_absolute_error: 2.4325 - lr: 2.5000e-04
Epoch 80/1000
1/1 [==============================] - 0s 34ms/step - loss: 21.0002 - mean_squared_error: 21.0002 - rmse: 4.5826 - mean_absolute_error: 3.4916 - val_loss: 9.7147 - val_mean_squared_error: 9.7147 - val_rmse: 3.1168 - val_mean_absolute_error: 2.4419 - lr: 2.5000e-04
Epoch 81/1000
1/1 [==============================] - 0s 34ms/step - loss: 19.8027 - mean_squared_error: 19.8027 - rmse: 4.4500 - mean_absolute_error: 3.6034 - val_loss: 9.7823 - val_mean_squared_error: 9.7823 - val_rmse: 3.1277 - val_mean_absolute_error: 2.4540 - lr: 2.5000e-04
Epoch 82/1000
1/1 [==============================] - 0s 42ms/step - loss: 18.6185 - mean_squared_error: 18.6185 - rmse: 4.3149 - mean_absolute_error: 3.4422 - val_

1/1 [==============================] - 0s 39ms/step - loss: 15.4790 - mean_squared_error: 15.4790 - rmse: 3.9343 - mean_absolute_error: 3.1702 - val_loss: 9.2167 - val_mean_squared_error: 9.2167 - val_rmse: 3.0359 - val_mean_absolute_error: 2.3436 - lr: 1.2500e-04
Epoch 108/1000
1/1 [==============================] - 0s 43ms/step - loss: 17.9399 - mean_squared_error: 17.9399 - rmse: 4.2356 - mean_absolute_error: 3.4709 - val_loss: 9.1531 - val_mean_squared_error: 9.1531 - val_rmse: 3.0254 - val_mean_absolute_error: 2.3300 - lr: 1.2500e-04
Epoch 109/1000
1/1 [==============================] - 0s 31ms/step - loss: 20.0619 - mean_squared_error: 20.0619 - rmse: 4.4791 - mean_absolute_error: 3.3879 - val_loss: 9.1042 - val_mean_squared_error: 9.1042 - val_rmse: 3.0173 - val_mean_absolute_error: 2.3194 - lr: 1.2500e-04
Epoch 110/1000
1/1 [==============================] - 0s 40ms/step - loss: 17.6002 - mean_squared_error: 17.6002 - rmse: 4.1953 - mean_absolute_error: 3.2095 - val_loss: 9.082

...

pooling layers: 4
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_20 (Conv1D)          (None, 100, 6)            1302      
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 50, 6)            0         
 g1D)                                                            
                                                                 
 conv1d_21 (Conv1D)          (None, 50, 16)            304       
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, 25, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_22 (Conv1D)          (None, 25, 2

1/1 [==============================] - 0s 33ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 5.0000e-04
Epoch 23/1000
1/1 [==============================] - 0s 25ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 5.0000e-04
Epoch 24/1000
1/1 [==============================] - 0s 33ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 5.0000e-04
Epoch 25/1000
1/1 [==============================] - 0s 29ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 5.0000e-04
Ep

Epoch 54/1000
1/1 [==============================] - 0s 27ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 2.5000e-04
Epoch 55/1000
1/1 [==============================] - 0s 28ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 2.5000e-04
Epoch 56/1000
1/1 [==============================] - 0s 37ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 2.5000e-04
Epoch 57/1000
1/1 [==============================] - 0s 46ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr:

Epoch 85/1000
1/1 [==============================] - 0s 31ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 6.2500e-05
Epoch 86/1000
1/1 [==============================] - 0s 31ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 6.2500e-05
Epoch 87/1000
1/1 [==============================] - 0s 30ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 6.2500e-05
Epoch 88/1000
1/1 [==============================] - 0s 26ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr:

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmpl6_k8xwy.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175, 100)